In [55]:
import os
from typing import List

import requests as requests
from typing import Union

In [68]:
def alchemy_asset_transfers(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str] = 'latest',
        page_key: str = None
):
    if not to_block == 'latest':
        to_block = hex(to_block)

    parameters = {
        "fromBlock": hex(from_block),
        "toBlock": to_block,
        "fromAddress": from_address,
        "toAddress": to_address
    }

    if not from_address:
        parameters.pop('fromAddress')
    if not to_address:
        parameters.pop('toAddress')

    if page_key:
        parameters['pageKey'] = page_key

    response_post = requests.post(
        "https://eth-mainnet.alchemyapi.io/v2/"
        + os.environ["ALCHEMY_API_KEY"],
        json={
            "jsonrpc": "2.0",
            "id": 0,
            "method": "alchemy_getAssetTransfers",
            "params": [parameters]
        },
    )

    response_json = response_post.json()
    return response_json

def get_lp_swaps(
        from_address: str,
        to_address: str,
        from_block: Union[int, str],
        to_block: Union[int, str]
) -> List:

    page_key = None
    all_txes = []
    while True:
        response = alchemy_asset_transfers(
            from_address=from_address,
            to_address=to_address,
            from_block=from_block,
            to_block=to_block,
            page_key=page_key
        )
        transactions = response["result"]["transfers"]
        all_txes.extend(transactions)
        if not 'pageKey' in response['result'].keys():
            break
        page_key = response['result']['pageKey']

    return all_txes

In [99]:
block_start = 13182518
block_end = 13206754
pool_addr = "0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7"

In [100]:
txes = get_lp_swaps(
    from_address=pool_addr, to_address='', from_block=block_start, to_block=block_end
)

In [101]:
txes[:5]

[{'blockNum': '0xc92644',
  'hash': '0xb344330689146906be75d8f6a1814d2d8442c66be3ae4bbbf716d8dfdbf55230',
  'from': '0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7',
  'to': '0x27239549dd40e1d60f5b80b0c4196923745b1fd2',
  'value': 494023.496198,
  'erc721TokenId': None,
  'erc1155Metadata': None,
  'asset': 'USDC',
  'category': 'token',
  'rawContract': {'value': '0x7306185206',
   'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
   'decimal': '0x6'}},
 {'blockNum': '0xc9264b',
  'hash': '0xe24619974f19e867c2fdb2901aed86e5b4c89e45cdecbb70a2bc603c6f1c3c7b',
  'from': '0xbebc44782c7db0a1a60cb6fe97d0b483032ff1c7',
  'to': '0x61e10659fe3aa93d036d099405224e4ac24996d0',
  'value': 499304.860926,
  'erc721TokenId': None,
  'erc1155Metadata': None,
  'asset': 'USDC',
  'category': 'token',
  'rawContract': {'value': '0x7440e38cfe',
   'address': '0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48',
   'decimal': '0x6'}},
 {'blockNum': '0xc9264f',
  'hash': '0xcd9626758a54d2d392a3e1721114f11489f54

In [102]:
len(txes)

1746

In [103]:
fraction_fee = 0.03 / 100

In [104]:
total_fees = 0
for i in txes:
    total_fees += i['value'] * fraction_fee
    block_num = i['blockNum']
    block_end = int(block_num, 16)

print(f"Fees collected by {pool_addr} from block {block_start} to {block_end} is {round(total_fees, 2)}.")

Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 13182518 to 13206749 is 145389.59.


In [113]:
blocks_to_query = [{
    "block": "12693255"
},
{
    "block": "12733780"
},
{
    "block": "12740342"
},
{
    "block": "12826286"
},
{
    "block": "12865685"
},
{
    "block": "12960904"
},
{
    "block": "12993519"
},
{
    "block": "13048460"
},
{
    "block": "13073420"
},
{
    "block": "13182518"
},
{
    "block": "13206754"
},
{
    "block": "13278344"
},
{
    "block": "13307602"
},
{
    "block": "13360861"
},
{
    "block": "13402155"
},
{
    "block": "13456217"
},
{
    "block": "13484749"
},
{
    "block": "13541142"
}
]
blocks = []
for i in blocks_to_query:
    blocks.append(int(i['block']))
blocks

[12693255,
 12733780,
 12740342,
 12826286,
 12865685,
 12960904,
 12993519,
 13048460,
 13073420,
 13182518,
 13206754,
 13278344,
 13307602,
 13360861,
 13402155,
 13456217,
 13484749,
 13541142]

In [115]:
block_ranges = list(zip(blocks, blocks[1:]))
block_ranges

[(12693255, 12733780),
 (12733780, 12740342),
 (12740342, 12826286),
 (12826286, 12865685),
 (12865685, 12960904),
 (12960904, 12993519),
 (12993519, 13048460),
 (13048460, 13073420),
 (13073420, 13182518),
 (13182518, 13206754),
 (13206754, 13278344),
 (13278344, 13307602),
 (13307602, 13360861),
 (13360861, 13402155),
 (13402155, 13456217),
 (13456217, 13484749),
 (13484749, 13541142)]

In [120]:
fees_for_range = {}
for (block_start, block_end) in block_ranges:

    txes = get_lp_swaps(
        from_address=pool_addr, to_address='', from_block=block_start, to_block=block_end
    )
    print(f"Block range {block_start}:{block_end} has {len(txes)} transactions for {pool_addr}")

    total_fees = 0
    for i in txes:
        total_fees += i['value'] * fraction_fee
        block_num = i['blockNum']
        last_block = int(block_num, 16)

    print(f"Fees collected by {pool_addr} from block {block_start} to {last_block} is {round(total_fees, 2)}.")

    fees_for_range[(block_start, block_end)] = total_fees

Block range 12693255:12733780 has 4370 transactions for 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7
Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 12693255 to 12733758 is 235315.84.
Block range 12733780:12740342 has 617 transactions for 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7
Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 12733780 to 12740339 is 51611.65.
Block range 12740342:12826286 has 7819 transactions for 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7
Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 12740342 to 12826277 is 486920.36.
Block range 12826286:12865685 has 3595 transactions for 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7
Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 12826286 to 12865674 is 275079.78.
Block range 12865685:12960904 has 9745 transactions for 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7
Fees collected by 0xbEbc44782C7dB0a1A60Cb6fe97d0b483032FF1C7 from block 1286568

In [121]:
fees_for_range

{(12693255, 12733780): 235315.84329081108,
 (12733780, 12740342): 51611.64508063634,
 (12740342, 12826286): 486920.3576488525,
 (12826286, 12865685): 275079.7803891132,
 (12865685, 12960904): 741996.076269439,
 (12960904, 12993519): 232767.91744170382,
 (12993519, 13048460): 379169.4613490703,
 (13048460, 13073420): 199588.39389553465,
 (13073420, 13182518): 574380.8313389953,
 (13182518, 13206754): 145389.58763483487,
 (13206754, 13278344): 479242.510586363,
 (13278344, 13307602): 252981.9117894561,
 (13307602, 13360861): 327884.333095976,
 (13360861, 13402155): 312335.41524615674,
 (13402155, 13456217): 569665.7341419263,
 (13456217, 13484749): 227896.96185654632,
 (13484749, 13541142): 645644.3463880663}